In [2]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Debug"
sys.path.append(atlas_path)


from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *
from AtlasPy.model import *

In [3]:
exchange_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/src/exchangeVBT"
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [4]:
st = time.perf_counter_ns()
n = 2
close = AssetReadNode.make("Close", 0, exchange)
open = AssetReadNode.make("Open", 0, exchange)
prev_close = AssetReadNode.make("Close", -1, exchange)

feat1 = AssetOpNode.make(
    close,
    prev_close,
    AssetOpType.SUBTRACT
)
feat2 = AssetOpNode.make(
    close,
    open,
    AssetOpType.SUBTRACT
)
target = ModelTarget(
    close,
    ModelTargetType.ABSOLUTE,
    2
)

In [5]:
config = ModelConfig(
    training_window = 5,
    walk_forward_window = 3,
    model_type = ModelType.LINEAR_REGRESSION,
    exchange = exchange,
)
lr_config = LinearRegressionModelConfig(
    config,
    LinearRegressionSolver.LDLT,
    True
)
lr_model = LinearRegressionModel(
    "lr_model",
    [feat1, feat2],
    target,
    lr_config
)
exchange.registerModel(lr_model)

for i in range(6):
    hydra.step()

In [17]:
ticker = "BTC-USD"
asset_id = exchange.getAssetIndex(ticker)
path = os.path.join(exchange_path,f"{ticker}.csv")  
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df["Date"]) 
df = df.set_index("Date")
df = df[['Open', 'High', 'Low', 'Close']]
df["feat_1"] = df["Close"] - df["Close"].shift(1)
df["feat_2"] = df["Close"] - df["Open"]
df["target"] = df["Close"].shift(-2)

df.head(10)

,Open,High,Low,Close,feat_1,feat_2,target
Date,,,,,,,
2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,NaN,-455.000000,15201.000000
2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,1324.899414,1357.099609,15599.200195
2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,218.900391,222.799805,17429.500000
2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,398.200195,328.500000,17527.000000
2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,1830.299805,1952.299805,16477.599609
2018-01-06 00:00:00+00:00,17462.099609,17712.400391,16764.599609,17527.000000,97.500000,64.900391,15170.099609
2018-01-07 00:00:00+00:00,17527.300781,17579.599609,16087.700195,16477.599609,-1049.400391,-1049.701172,14595.400391
2018-01-08 00:00:00+00:00,16476.199219,16537.900391,14208.200195,15170.099609,-1307.500000,-1306.099609,14973.299805
2018-01-09 00:00:00+00:00,15123.700195,15497.500000,14424.000000,14595.400391,-574.699219,-528.299805,13405.799805


In [6]:
x = lr_model.getX()
y = lr_model.getY()
theta = lr_model.getTheta()
x,y,theta

(array([[1.32489941e+03, 1.35709961e+03, 1.00000000e+00],
        [2.18900391e+02, 2.22799805e+02, 1.00000000e+00],
        [3.98200195e+02, 3.28500000e+02, 1.00000000e+00],
        [1.83029980e+03, 1.95229980e+03, 1.00000000e+00],
        [9.75000000e+01, 6.49003906e+01, 1.00000000e+00]]),
 array([15599.20019531, 17429.5       , 17527.        ,     0.        ,
            0.        ]),
 array([ 3.51611216e+00, -5.04197955e+00,  1.77831737e+04]))

In [7]:
hydra.step()
x = lr_model.getX()
y = lr_model.getY()
theta = lr_model.getTheta()

model = sm.OLS(y[1:4], x[1:4,:]).fit()
params = np.array(model.params)

x,y,theta,params

(array([[-1.04940039e+03, -1.04970117e+03,  1.00000000e+00],
        [ 2.18900391e+02,  2.22799805e+02,  1.00000000e+00],
        [ 3.98200195e+02,  3.28500000e+02,  1.00000000e+00],
        [ 1.83029980e+03,  1.95229980e+03,  1.00000000e+00],
        [ 9.75000000e+01,  6.49003906e+01,  1.00000000e+00]]),
 array([15599.20019531, 17429.5       , 17527.        , 16477.59960938,
            0.        ]),
 array([ 1.92627193e+00, -2.34512509e+00,  1.75303317e+04]),
 array([ 1.92627193e+00, -2.34512509e+00,  1.75303317e+04]))

In [8]:
hydra.step()
x = lr_model.getX()
y = lr_model.getY()
theta = lr_model.getTheta()
x,y,theta

(array([[-1.04940039e+03, -1.04970117e+03,  1.00000000e+00],
        [-1.30750000e+03, -1.30609961e+03,  1.00000000e+00],
        [ 3.98200195e+02,  3.28500000e+02,  1.00000000e+00],
        [ 1.83029980e+03,  1.95229980e+03,  1.00000000e+00],
        [ 9.75000000e+01,  6.49003906e+01,  1.00000000e+00]]),
 array([15599.20019531, 17429.5       , 17527.        , 16477.59960938,
        15170.09960938]),
 array([ 1.92627193e+00, -2.34512509e+00,  1.75303317e+04]))

In [9]:
hydra.step()
x = lr_model.getX()
y = lr_model.getY()
theta = lr_model.getTheta()
x,y,theta

(array([[-1.04940039e+03, -1.04970117e+03,  1.00000000e+00],
        [-1.30750000e+03, -1.30609961e+03,  1.00000000e+00],
        [-5.74699219e+02, -5.28299805e+02,  1.00000000e+00],
        [ 1.83029980e+03,  1.95229980e+03,  1.00000000e+00],
        [ 9.75000000e+01,  6.49003906e+01,  1.00000000e+00]]),
 array([14595.40039062, 17429.5       , 17527.        , 16477.59960938,
        15170.09960938]),
 array([ 1.92627193e+00, -2.34512509e+00,  1.75303317e+04]))

In [27]:
#model = sm.OLS(y[1:4], x[1:4,:]).fit()
#print(model.params)
df.head(10)

,Open,High,Low,Close,feat_1,feat_2,target
Date,,,,,,,
2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,NaN,-455.000000,15201.000000
2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,1324.899414,1357.099609,15599.200195
2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,218.900391,222.799805,17429.500000
2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,398.200195,328.500000,17527.000000
2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,1830.299805,1952.299805,16477.599609
2018-01-06 00:00:00+00:00,17462.099609,17712.400391,16764.599609,17527.000000,97.500000,64.900391,15170.099609
2018-01-07 00:00:00+00:00,17527.300781,17579.599609,16087.700195,16477.599609,-1049.400391,-1049.701172,14595.400391
2018-01-08 00:00:00+00:00,16476.199219,16537.900391,14208.200195,15170.099609,-1307.500000,-1306.099609,14973.299805
2018-01-09 00:00:00+00:00,15123.700195,15497.500000,14424.000000,14595.400391,-574.699219,-528.299805,13405.799805


In [11]:
hydra.step()
x = lr_model.getX()
y = lr_model.getY()
theta = lr_model.getTheta()
x,y,theta

(array([[-1.04940039e+03, -1.04970117e+03,  1.00000000e+00],
        [-1.30750000e+03, -1.30609961e+03,  1.00000000e+00],
        [-5.74699219e+02, -5.28299805e+02,  1.00000000e+00],
        [ 3.77899414e+02,  3.84799805e+02,  1.00000000e+00],
        [ 9.75000000e+01,  6.49003906e+01,  1.00000000e+00]]),
 array([14595.40039062, 14973.29980469, 17527.        , 16477.59960938,
        15170.09960938]),
 array([  -23.4274429 ,    24.62193088, 15856.30236006]))

In [25]:
x_pandas = df[["feat_1","feat_2"]].iloc[5:8].values
x_pandas = sm.add_constant(x_pandas)
x_pandas = x_pandas[:, [2, 1, 0]]
y_pandas = df["target"].iloc[5:8].values
x_pandas

array([[ 9.75000000e+01,  6.49003906e+01,  1.00000000e+00],
       [-1.04940039e+03, -1.04970117e+03,  1.00000000e+00],
       [-1.30750000e+03, -1.30609961e+03,  1.00000000e+00]])

In [26]:
model = sm.OLS(y_pandas, sm.add_constant(x_pandas)).fit()
params = np.array(model.params)
params

array([   89.04273578,   -91.10739896, 12401.33865168])

In [16]:

x_pandas, y_pandas

(array([[   97.5       ,    64.90039062],
        [-1049.40039062, -1049.70117188],
        [-1307.5       , -1306.09960938]]),
 array([15170.09960938, 14595.40039062, 14973.29980469]))

In [14]:
"""
ticker = "BTC-USD"
asset_id = exchange.getAssetIndex(ticker)
path = os.path.join(exchange_path,f"{ticker}.csv")  
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df["Date"]) 
df = df.set_index("Date")

def ts_argmax(df, window=10):
    return df.rolling(window).apply(np.argmax).add(1)

btc_idx = exchange.getAssetIndex("BTC-USD")
df["close_arg_max_atlas"] = close_arg_max.cache()[btc_idx].T
df["close_max_atlas"] = close_max.cache()[btc_idx].T
df["close_max_pd"] = df["Close"].diff().rolling(n).max()
df["close_arg_max_pd"] = ts_argmax(df["Close"].diff(), n)
df["change"] = df["Close"].diff()
df = df[["Close","change", "close_max_atlas", "close_max_pd", "close_arg_max_pd", "close_arg_max_atlas"]]
df.head(30)
"""

'\nticker = "BTC-USD"\nasset_id = exchange.getAssetIndex(ticker)\npath = os.path.join(exchange_path,f"{ticker}.csv")  \ndf = pd.read_csv(path)\ndf["Date"] = pd.to_datetime(df["Date"]) \ndf = df.set_index("Date")\n\ndef ts_argmax(df, window=10):\n    return df.rolling(window).apply(np.argmax).add(1)\n\nbtc_idx = exchange.getAssetIndex("BTC-USD")\ndf["close_arg_max_atlas"] = close_arg_max.cache()[btc_idx].T\ndf["close_max_atlas"] = close_max.cache()[btc_idx].T\ndf["close_max_pd"] = df["Close"].diff().rolling(n).max()\ndf["close_arg_max_pd"] = ts_argmax(df["Close"].diff(), n)\ndf["change"] = df["Close"].diff()\ndf = df[["Close","change", "close_max_atlas", "close_max_pd", "close_arg_max_pd", "close_arg_max_atlas"]]\ndf.head(30)\n'